In [28]:
from gensim.models import word2vec
import json
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
m = MorphAnalyzer()

278677it [22:00, 211.01it/s]


Для задачи выделения именованных сущностей я взяла датасет про одежду, обувь и украшения. Предварительно сделала токенизацию и лемматизацию:

In [ ]:
lemm_texts = []
for line in tqdm(open('reviews.json', 'r')):
    tok_text = word_tokenize(json.loads(line)['reviewText'])
    lemmas = [m.parse(word)[0].normal_form for word in tok_text if word.isalpha()]
    lemm_texts.append(lemmas)

Три способа выделить именованные сущности:
1. Составить правила. Достоинства: можем четко задать правила, и не получить лишние сущности. Недостаток метода состоит в том, что правила необходимо прописывать вручную, и это занимает определенное количество времени. Кроме того, название брендов и дескрипторов как правило достаточно разнообразные, и их будет достаточно сложно выделять с помощью правил.
2. Классификационный метод. Можем выделить в тексте всех кандидатов (например все ИГ), выделить для них признаки и классифицировать по ним. Недостаток метода - нужны размеченные данные, которых у нас нет
3. Составить список базовых дескрипторов и брендов и расширить его с помощью эмбеддингов. Недостаток - можем получить лишние сущности, если плохо подберем порог близости. Этот метод хорошо нам подходит, поскольку можно будет составить список базовых дескрипторов, и затем расширить его, покрыв все поле.

Воспользуемся третьим методом

In [29]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=lemm_texts, vector_size=100)

Составим список дескрипторов и с помощью Word2vec найдем для каждого 5 ближайших слов. Также составим список оценочных прилагательных, и для каждого возьмем по 10 соседей, чтобы выделять именно коллокации с оценкой: 

In [66]:
descr = ['skirt', 'dress', 'jeans', 'jacket', 'shirt', 'socks', 'leggings', 'underwear', 'sweater', 'top',
        'boots', 'shoes', 'sneakers', 'necklace', 'earrings']
all_descr = []
for word in descr:
    if word not in all_descr:
        all_descr.append(word)
    for sim_word in model.wv.most_similar(word, topn=5):
        if sim_word[0] not in all_descr:
            all_descr.append(sim_word[0])
eval = ['beautiful', 'comfortable', 'stylish', 'nice', 'uncomfortable', 'strange', 'unattractive']
all_eval = []
for word in eval:
    if word not in all_eval:
        all_eval.append(word)
    for sim_word in model.wv.most_similar(word, topn=10):
        if sim_word[0] not in all_eval:
            all_eval.append(sim_word[0])

In [67]:
#Выделим все биграммы с необходимыми словами
bigrams = []
for text in lemm_texts:
    for ind, word in enumerate(text):
        if word in all_descr:
            if ind != 0:
                bigrams.append((text[ind-1], word))
            if ind != len(text) - 1:
                bigrams.append((word, text[ind+1]))

In [68]:
#Найдем коллокации, для отчета будем использовать pmi
import nltk
from nltk.collocations import BigramCollocationFinder
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(lemm_texts)
finder.apply_freq_filter(10)
pmi_coll = finder.score_ngrams(bigram_measures.pmi)
log_coll = finder.score_ngrams(bigram_measures.likelihood_ratio)
chi_coll = finder.score_ngrams(bigram_measures.chi_sq)

In [77]:
#берем коллокации, найденные с помощью pmi, выбираем среди них те, 
#которые относятся к нашим товарам, а также содержат оценочные прилагательные
goods_coll = list(set(bigrams) & set([coll[0] for coll in pmi_coll]))
final_coll = {}
for coll in goods_coll:
    descr_word = set(all_descr) & set(coll)
    eval_word = set(all_eval) & set(coll)
    if len(eval_word) != 0:
        d_word = list(descr_word)[0]
        if d_word not in final_coll:
            final_coll[d_word] = []
        final_coll[d_word].append(coll)

In [78]:
for good in list(final_coll)[:5]:
    print(good)
    print(final_coll[good])


sandals
[('nice', 'sandals'), ('pretty', 'sandals'), ('cute', 'sandals'), ('great', 'sandals'), ('comfortable', 'sandals'), ('comfy', 'sandals'), ('sandals', 'great'), ('good', 'sandals')]
socks
[('wonderful', 'socks'), ('good', 'socks'), ('nice', 'socks'), ('lightweight', 'socks'), ('socks', 'great'), ('tight', 'socks'), ('great', 'socks'), ('cute', 'socks'), ('decent', 'socks'), ('comfortable', 'socks'), ('socks', 'nice'), ('comfy', 'socks'), ('socks', 'good')]
boxers
[('great', 'boxers'), ('comfortable', 'boxers')]
jeans
[('tight', 'jeans'), ('comfy', 'jeans'), ('jeans', 'great'), ('good', 'jeans'), ('nice', 'jeans'), ('great', 'jeans'), ('cute', 'jeans'), ('jeans', 'nice'), ('jeans', 'good'), ('comfortable', 'jeans')]
blouse
[('lovely', 'blouse'), ('nice', 'blouse'), ('beautiful', 'blouse'), ('pretty', 'blouse'), ('cute', 'blouse'), ('great', 'blouse')]
